<div dir=rtl>

# کتابخانه‌های مورد نیاز

</div>

In [4]:
import pandas as pd
import networkx as nx
import numpy as np
import sys

<div dir=rtl>

# کلاس گره‌های گراف

</div>

<div dir=rtl>

در این قسمت کلاسی که ساختار گره‌های گراف را نگهداری می‌کند را ایجاد می‌کنیم. در این کلاس برای هر گره یک شناسه که همان شماره گره است، مقدار بالانس که همان مقدار b است که در مساله برای هر گره تعریف می شود و یال‌هایی که مربوط به آن گره می‌باشند.

در این کلاس متودهای update potential و add edges وجود دارند که به ترتیب مقادیر پتانسیل گره‌ها را بروزرسانی می‌کند (توجه کنید که مقدار اولیه این برای تمام گره‌ها صفر می‌باشد.)  و
برای هر یال:
اگر شناسه گره فعلی با شناسه گره مقصد یال یکی نباشد، یال به دیکشنری edges گره اضافه می‌شود.

In [5]:
class Node:
    potentialValue = 0
    def __init__(self, nodeID, balance, edges):
        self.nodeID = nodeID
        self.balance = balance
        self.edges = edges

    def updatePotential(self, potentialValue):
        self.potentialValue = potentialValue

    def addEdges(self, *edges):
        for edge in edges:
            if self.nodeID != edge.desNode.nodeID:
                self.edges[edge.desNode.nodeID] = edge

    def printNode(self, printAllEdge = False):
        print("node "+str(self.nodeID) + ", balance: " + str(self.balance) +
              ", potential:"+ str(self.potentialValue))
        if printAllEdge == True:
            for edge in self.edges.values():
                print("destination:" + str(edge.desNode.nodeID) + " residualCapacity: "+ str(edge.residualCapacity)
                      + ", reducedCost: "+ str(edge.reducedCost) + ", cost: " + str(edge.cost))

<div dir=rtl>

# کلاس یال‌های گراف

</div>

<div dir=rtl>

این کلاس یک ساختار پایه برای یال‌ها در یک گراف را فراهم می‌کند . هر یال دارای مقادیر مختلفی است از جمله:

desNode: گره مقصد.

cost: هزینه عبور از یال.

capacity: ظرفیت یال.

residualCapacity: ظرفیت باقیمانده.

reducedCost: هزینه کاهش یافته.

flow = 0: این یک متغیر کلاس است که مقدار اولیه 0 دارد و جریان یال را نشان می‌دهد. این متغیر به طور پیش‌فرض برای تمام نمونه‌های این کلاس تنظیم می‌شود، اما در صورت نیاز می‌تواند در هر نمونه تغییر یابد.

متدهای این کلاس به ما اجازه می‌دهند که ظرفیت باقیمانده و هزینه کاهش یافته یال را به روز کنیم.

</div>

In [6]:
class Edge:
    flow = 0
    def __init__(self, desNode, cost, capacity):
        self.desNode = desNode
        self.cost = cost
        self.capacity = capacity
        self.residualCapacity = capacity
        self.reducedCost = cost

    def updateResidualCapacity(self, residualCapacity):
        self.residualCapacity = residualCapacity

    def updateReducedCost(self, reducedCost):
        self.reducedCost = reducedCost

<div dir=rtl>

# توابع مربوط به الگوریتم دایکسترا برای پیدا کردن کوتاه‌ترین مسیرها

</div>

<div dir=rtl>

در این سل و سل بعدی الگوریتم دایکسترا و کلاس کمکی صف اولویت پیاده سازی شده است. این الگوریتم در نهایت کوتاه‌ترین فاصله‌ی root داده شده را با تمام  nodes برمیگرداند.

 نکته‌ای که درمورد این الگوریتم وجود دارد آن است که فقط برای طول یال مثبت کار می کند و به همین دلیل ما از reduced cost برای مثبت نگه داشتن این مقادیر استفاده می‌کنیم. (تا در واقع بتوانیم از این الگوریتم به جای بلمن فورد (که در آن شرطی روی مثبت بودن یال‌ها نیست ولی پیچیدگی زمانی‌اش از دایکسترا بیشتر است.) استفاده کنیم.

### توضیح خلاصه الگوریتم دیکسترا:

الگوریتم دیکسترا برای یافتن کوتاه‌ترین مسیرها از یک گره مبدا (ریشه) به تمام گره‌های دیگر در یک گراف وزن‌دار و جهت‌دار استفاده می‌شود. در این الگوریتم:

مقداردهی اولیه:
هزینه رسیدن به هر گره از گره ریشه برابر با بی‌نهایت (sys.maxsize) تنظیم می‌شود، به جز هزینه رسیدن به گره ریشه که صفر است.

یک صف اولویت برای نگهداری گره‌هایی که باید بازدید شوند، با اولویت بر اساس هزینه فعلی شناخته شده، ایجاد می‌شود.

### حلقه اصلی:
تا زمانی که صف اولویت خالی نشده است، گره‌ای که کمترین هزینه فعلی را دارد از صف خارج می‌شود و به عنوان بازدید شده علامت‌گذاری می‌شود.

برای هر یال خروجی از گره فعلی، اگر گره مقصد بازدید نشده باشد، هزینه کاهش یافته آن کمتر از هزینه فعلی شناخته شده باشد و ظرفیت باقیمانده مثبت باشد، هزینه و گره قبلی به‌روزرسانی می‌شود.

اگر گره مقصد در صف نباشد، به صف اضافه می‌شود.

### خروجی:

لیستی از هزینه‌ها که شامل هزینه کوتاه‌ترین مسیر از گره ریشه به سایر گره‌ها و گره قبلی در مسیر کوتاه‌ترین است، بازگردانده می‌شود.
### پیچیدگی زمانی:

پیچیدگی زمانی الگوریتم دیکسترا اگر از یک صف اولویت کارآمد
که  استفاده شود، پیچیدگی زمانی به صورت زیر خواهد بود:

O((V + E) log V)

که در آن:

V تعداد گره‌ها (رئوس) در گراف است.

E تعداد یال‌ها (لبه‌ها) در گراف است.

</div>

In [7]:
class PriorityQueue:

    def __init__(self,element):
        self.queue = [element]

    def add(self, element):
        i = 0

        if self.isEmpty():
            self.queue = [element]
            return

        while i < len(self.queue) and element[1] < self.queue[i][1]:
            i += 1

        self.queue.insert(i, element)

    def pop(self):
        return self.queue.pop()[0]

    def contains(self,element):
        return element in self.queue

    def isEmpty(self):
        return len(self.queue) == 0

In [8]:
def dijkstra(nodes,root):
      costs = []  #save the costs from root to other nodes
      #The list is initialized with tuples, where the first element of each tuple represents the cost, and the second element represents the previous node  in the shortest path.
      for i in range(len(nodes)):
          costs.append((sys.maxsize, root)) # sys.maxsize == infinity

      costs[root] = (0, root) #sets the cost of the root node to zero since the cost to reach the root from itself is zero.
      visited = [False for i in range(len(nodes))]
      # queue contains open nodes to visit
      queue = PriorityQueue((root,0)) #The priority queue is used to keep track of the nodes that need to be visited, with priorities based on the current known costs.

      # update costs while queue is not empty
      while not queue.isEmpty():
          node = queue.pop()
          visited[node] = True

          # expand node and update path costs
          for edge in nodes[node].edges.values():
              if not visited[edge.desNode.nodeID] \
                      and edge.reducedCost + costs[node][0] < costs[edge.desNode.nodeID][0] \
                      and edge.residualCapacity > 0:
                  costs[edge.desNode.nodeID] = (edge.reducedCost + costs[node][0], node)

                  if not queue.contains(edge.desNode.nodeID):
                      queue.add((edge.desNode.nodeID,costs[edge.desNode.nodeID]))
      return costs

<div dir=rtl>

# کلاس گراف

</div>

<div dir=rtl>

کلاس Graph یک چارچوب کامل برای مدیریت و تحلیل یک گراف جریان فراهم می‌کند. این کلاس شامل متدهایی برای به‌روزرسانی هزینه‌ها و جریان‌ها، یافتن مسیرهای کوتاه، اضافه کردن منابع و مقصدهای ساختگی، و وارد کردن داده‌ها از فایل اکسل است.

nodes: دیکشنری‌ای که شامل تمام گره‌های گراف است.

excess: مجموعه‌ای از گره‌های منبع که تعادل آن‌ها مثبت است.

deficit: مجموعه‌ای از گره‌های مقصد که تعادل آن‌ها منفی است.

 متد update reduced cost هزینه کاهش یافته هر یال را بر اساس پتانسیل گره‌ها به‌روزرسانی می‌کند.

  متد update flow جریان را در یال‌های مسیر مشخص به‌روزرسانی می‌کند و ظرفیت باقیمانده و جریان معکوس را نیز تنظیم می‌کند.

   متد update balance بالانس گره‌های مبدأ و مقصد را بر اساس جریان به‌روزرسانی می‌کند.

   این متد update Residual Network شبکه باقی‌مانده را به‌روزرسانی می‌کند و اگر یک یال معکوس وجود ندارد، آن را اضافه می‌کند.

   این متد print Flow جریان‌های مثبت در یال‌ها را چاپ می‌کند.

   این متد find Path از الگوریتم دیکسترا برای پیدا کردن مسیر کوتاه‌ترین استفاده می‌کند و سپس مسیر را برمی‌گرداند.

   این متد input data from excell داده‌ها را از یک فایل اکسل خوانده و گره‌ها و یال‌ها را به گراف اضافه می‌کند.

   این متد add Dummy Source یک گره منبع ساختگی s به گراف اضافه می‌کند.

   این متد add Dummy sink یک گره مقصد ساختگی t به گراف اضافه می‌کند.

</div>

<div dir=rtl>

### متد successive shortest path

این متد الگوریتم اصلی ما در این روش می‌باشد که در آن با استفاده از الگوریتم SSP مقدار minimum cost flow را محاسبه می کنیم.

این الگوریتم به طور مکرر از الگوریتم دیکسترا برای یافتن کوتاه‌ترین مسیرها در شبکه استفاده می‌کند و سپس جریان را به‌روزرسانی می‌کند تا زمانی که تمام جریان از منبع به مقصد ارسال شود. پیچیدگی زمانی الگوریتم به مقدار جریان کل و تعداد گره‌ها و یال‌ها وابسته است و در بدترین حالت
𝑂
(
𝐶
⋅
(
𝑉
+
𝐸
)
log
⁡
𝑉
)
 خواهد بود.

 الگوریتم successiveShortestPathAlgorithm یک روش برای حل مسئله جریان کم‌هزینه (minimum cost flow) در شبکه‌های جریان است. این الگوریتم به‌طور خلاصه به صورت زیر عمل می‌کند:

حلقه اصلی:

تا زمانی که تراز گره‌های منبع و مقصد صفر نشده است (یعنی همه جریان مورد نیاز از منبع به مقصد ارسال نشده است)، مراحل زیر تکرار می‌شوند:

یافتن مسیر:

 از الگوریتم دیکسترا برای یافتن کوتاه‌ترین مسیر از منبع به مقصد استفاده می‌شود.

محاسبه جریان حداقل:

 جریان حداقل
(min flow) به عنوان کوچک‌ترین مقدار بین تراز منبع، تراز مقصد و ظرفیت باقیمانده در مسیر یافت‌شده محاسبه می‌شود.

به‌روزرسانی هزینه کاهش یافته:

هزینه‌های کاهش یافته یال‌ها به‌روز می‌شوند.


به‌روزرسانی جریان:

 جریان در طول مسیر به‌روزرسانی می‌شود و ظرفیت باقیمانده کاهش می‌یابد.

به‌روزرسانی تراز:

 تراز گره‌های منبع و مقصد به‌روزرسانی می‌شود.

به‌روزرسانی شبکه باقیمانده:

 شبکه باقیمانده (residual network) به‌روزرسانی می‌شود.

محاسبه هزینه نهایی:

پس از اتمام حلقه اصلی، هزینه نهایی جریان محاسبه و چاپ می‌شود.

پیچیدگی زمانی:

پیچیدگی زمانی الگوریتم successiveShortestPathAlgorithm به صورت زیر است:

یافتن کوتاه‌ترین مسیر: استفاده از الگوریتم دیکسترا در هر تکرار با پیچیدگی
𝑂
(
  𝑉
)


𝑂
(
(
𝑉
+
𝐸
)
log
⁡
𝑉
)

به‌روزرسانی جریان و ظرفیت‌ها: هر تکرار شامل به‌روزرسانی جریان، ظرفیت‌ها، تراز و شبکه باقیمانده است که هر یک از این عملیات‌ها در زمان
𝑂
(
𝐸
)
انجام می‌شوند.
تعداد تکرارهای حلقه اصلی در بدترین حالت برابر با مقدار جریان کل
𝐶
 است. بنابراین، پیچیدگی زمانی کلی الگوریتم به صورت زیر خواهد بود:

𝑂
(
𝐶
⋅
(
𝑉
+
𝐸
)
log
⁡
𝑉
)

که در آن
𝐶
 مقدار جریان کل است.

</div>

In [9]:
class Graph:
    nodes = {}
    excess = set()
    deficit = set()
    def __init__(self, nodeList = None):
        if nodeList == None: return
        for node in nodeList:
            self.nodes[node.nodeID] = node
            if node.balance > 0:
                self.excess.add(node.nodeID)
            elif node.balance < 0:
                self.deficit.add(node.nodeID)

    def updateReducedCost(self):
        for node in self.nodes.values():
            for edge in node.edges.values():
                if edge.residualCapacity > 0:
                    reducedCost = edge.cost - node.potentialValue + edge.desNode.potentialValue
                    edge.updateReducedCost(reducedCost)

    def updateFlow(self ,flow, path):
        for idx in range(len(path)-1):
          presentNode = path[idx]
          afterNode = path[idx + 1]
          edge = self.nodes[presentNode].edges[afterNode]
          edge.residualCapacity = edge.residualCapacity - flow
          edge.flow = edge.flow + flow

          if presentNode in self.nodes[afterNode].edges:
            reverseEdge = self.nodes[afterNode].edges[presentNode]
            reverseEdge.flow = reverseEdge.flow - flow
            reverseEdge.residualCapacity += flow

    def updateBalance(self, root, end, flow):
        self.nodes[root].balance -= flow
        self.nodes[end].balance += flow

    def updateResidualNetwork(self, path):
      for index in range(len(path) - 1):
        presentNode = path[index]
        afterNode = path[index + 1]
        if not (path[index] in self.nodes[path[index + 1]].edges):
          # Add edge
           newEdge = Edge(self.nodes[presentNode], -(self.nodes[presentNode].edges[afterNode].cost), self.nodes[presentNode].edges[afterNode].flow)
           newEdge.reducedCost =  -(self.nodes[presentNode].edges[afterNode].reducedCost)
           self.nodes[afterNode].addEdges(newEdge)

    def printFlow(self):
        for node in self.nodes.values():
          for edge in node.edges.values():
            if edge.cost > 0:
              print(str(node.nodeID) + "->" + str(edge.desNode.nodeID) + "\t Flow: " +str(edge.flow))

    def findPath(self):
        # apply Dijkstra algorithm
        costs = dijkstra(self.nodes, 0)
        end=len(self.nodes) -1
        path = [end]
        while end != 0:
          path.append(costs[end][1])
          end = costs[end][1]
        for i in range(1, len(self.nodes)):
          if costs[i][0] != sys.maxsize:
            self.nodes[i].potentialValue -= costs[i][0]
        if path:
          path.reverse()
          return path

    def findPathForCS(self, delta):
    # apply Dijkstra algorithm
      self.updateReducedCost()
      costs = dijkstra(self.nodes, 0)
      end = len(self.nodes) - 1
      if costs[end][0] == sys.maxsize:
        return None

      path = [end]
      while end != 0:
        path.append(costs[end][1])
        end = costs[end][1]

      path.reverse()
      return path

    def succesiveShortestPathAlgorithm(self):
      while(self.nodes[0].balance != 0 and self.nodes[len(self.nodes) - 1].balance !=0 ):
        path = self.findPath()

        minCapacity = sys.maxsize
        for i in range(len(path)-1):
          residual = self.nodes[path[i]].edges[path[i+1]].residualCapacity
          if residual < minCapacity:
            minCapacity = residual
        flow = min(abs(self.nodes[path[0]].balance), abs(self.nodes[path[-1]].balance), minCapacity)

        self.updateReducedCost()
        self.updateFlow(flow,path)
        self.updateBalance(path[0], path[-1], flow)
        self.updateResidualNetwork(path)
        self.printGraph()
        print("\n\n")

      cost = 0
      for node in self.nodes.values():
        for edge in node.edges.values():
          if edge.cost > 0:
            cost += edge.cost * edge.flow

      print("Minimum cost of succesiveShortestPathAlgorithm: " + str(cost))
      self.printFlow()

    def capacityScalingAlgorithm(self):
      maxCapacity = max(edge.capacity for node in self.nodes.values() for edge in node.edges.values())
      delta = 1
      while delta <= maxCapacity:
        delta <<= 1

      while delta > 0:
        self.updateReducedCost()
        path = self.findPathForCS(delta)
        while path:
          minCapacity = sys.maxsize
          for i in range(len(path) - 1):
            residual = self.nodes[path[i]].edges[path[i + 1]].residualCapacity
            if residual < minCapacity:
              minCapacity = residual
          flow = min(abs(self.nodes[path[0]].balance), abs(self.nodes[path[-1]].balance), minCapacity)

          self.updateFlow(flow, path)
          self.updateBalance(path[0], path[-1], flow)
          self.updateResidualNetwork(path)
          self.printGraph()
          print("\n\n")
          path = self.findPathForCS(delta)
        delta >>= 1

      cost = 0
      for node in self.nodes.values():
        for edge in node.edges.values():
          if edge.cost > 0:
            cost += edge.cost * edge.flow

      print("Minimum cost of capacity scaling: " + str(cost))
      self.printFlow()

    def inputDataFromExcel(self, filepath):
      nodesList = []
      nodeFile = pd.read_excel(filepath, sheet_name='Node')
      edgeFile = pd.read_excel(filepath, sheet_name='Edge')
      for index, row in nodeFile.iterrows():
        nodesList.append(Node(int(row['ID']), int(row['Balance']), {}))
      for node in nodesList:
            self.nodes[node.nodeID] = node
            if node.balance > 0:
                self.excess.add(node.nodeID)
            elif node.balance < 0:
                self.deficit.add(node.nodeID)
      for index, row in edgeFile.iterrows():
        edge = Edge(self.nodes[row['end_ID']], row['cost'], row['capacity'])
        self.nodes[row['start_ID']].addEdges(edge)

    def addDummySource(self):
      dummySource = Node(0, 0, {}) # initialize balance =0
      self.nodes[0]= dummySource
      for nodeID in self.excess:
        self.nodes[0].balance+= self.nodes[nodeID].balance
        edge=Edge(self.nodes[nodeID], 0, self.nodes[nodeID].balance)
        self.nodes[0].addEdges(edge)

    def addDummySink(self):
      dummySink = Node(len(self.nodes), 0, {}) # initialize balance =0
      for nodeID in self.deficit:
        dummySink.balance+= self.nodes[nodeID].balance
        edge=Edge(dummySink, 0, -self.nodes[nodeID].balance)
        self.nodes[nodeID].addEdges(edge)
      self.nodes[len(self.nodes)]= dummySink

    def printGraph(self):
        strExcess = "Excess:"
        strDeficit = "Deficit:"
        for nodeID in self.excess:
            strExcess += str(nodeID) +", "
        for nodeID in self.deficit:
            strDeficit +=  str(nodeID) +", "
        print(strExcess)
        print(strDeficit)
        for node in self.nodes.values():
            node.printNode(True)

In [10]:
myGraph=Graph()
filepath="/content/mm_0 copy.xlsx"
myGraph.inputDataFromExcel(filepath)
myGraph.addDummySource()
myGraph.addDummySink()
myGraph.succesiveShortestPathAlgorithm()

Excess:1, 
Deficit:4, 
node 1, balance: 4, potential:0
destination:2 residualCapacity: 4, reducedCost: 0, cost: 2
destination:3 residualCapacity: 0, reducedCost: 0, cost: 2
destination:0 residualCapacity: 2, reducedCost: 0, cost: 0
node 2, balance: 0, potential:-2
destination:4 residualCapacity: 3, reducedCost: 2, cost: 3
destination:3 residualCapacity: 2, reducedCost: 1, cost: 1
node 3, balance: 0, potential:-2
destination:4 residualCapacity: 3, reducedCost: 0, cost: 1
destination:1 residualCapacity: 2, reducedCost: 0, cost: -2
node 4, balance: -4, potential:-3
destination:5 residualCapacity: 2, reducedCost: 0, cost: 0
destination:3 residualCapacity: 2, reducedCost: 0, cost: -1
node 0, balance: 2, potential:0
destination:1 residualCapacity: 2, reducedCost: 0, cost: 0
node 5, balance: -2, potential:-3
destination:4 residualCapacity: 2, reducedCost: 0, cost: 0



Excess:1, 
Deficit:4, 
node 1, balance: 4, potential:0
destination:2 residualCapacity: 2, reducedCost: 0, cost: 2
destination

In [11]:
myGraph=Graph()
filepath="/content/mm_0 copy.xlsx"
myGraph.inputDataFromExcel(filepath)
myGraph.addDummySource()
myGraph.addDummySink()
myGraph.capacityScalingAlgorithm()

Excess:1, 
Deficit:4, 
node 1, balance: 4, potential:0
destination:2 residualCapacity: 4, reducedCost: 2, cost: 2
destination:3 residualCapacity: 0, reducedCost: 2, cost: 2
destination:0 residualCapacity: 2, reducedCost: 0, cost: 0
node 2, balance: 0, potential:0
destination:4 residualCapacity: 3, reducedCost: 3, cost: 3
destination:3 residualCapacity: 2, reducedCost: 1, cost: 1
node 3, balance: 0, potential:0
destination:4 residualCapacity: 3, reducedCost: 1, cost: 1
destination:1 residualCapacity: 2, reducedCost: -2, cost: -2
node 4, balance: -4, potential:0
destination:6 residualCapacity: 2, reducedCost: 0, cost: 0
destination:3 residualCapacity: 2, reducedCost: -1, cost: -1
node 0, balance: 2, potential:0
destination:1 residualCapacity: 2, reducedCost: 0, cost: 0
node 5, balance: 0, potential:-4
destination:4 residualCapacity: 4, reducedCost: 0, cost: 0
node 6, balance: -2, potential:0
destination:4 residualCapacity: 2, reducedCost: 0, cost: 0



Excess:1, 
Deficit:4, 
node 1, bala